In [3]:
import pandas as pd
import numpy as np
import math

In [8]:
df_crc = pd.read_csv('/localhome/cschiebroek/ShuZe/vp/data cleaning/crc_handbook_subset.tsv', sep='\t')
df_crc['VP_Pa'] = df_crc['Vapor pressure, kPa'] * 1000
df_crc['log10_VP_Pa'] = df_crc['VP_Pa'].apply(lambda x: math.log10(x))

,Unnamed: 0,"Mass density temperature, K","Mass density, kg/m3","Vapor pressure, kPa",canonical_smiles,cas_id,ref_id,VP_Pa,log10_VP_Pa
0,0,293.15,713.6,2.680,C-C-C-C(-C-C)-C-C,619-99-8,RU18.1_5044,2680.0,3.428135
1,1,298.15,907.7,0.245,C=C-C1:C:C:C:C:C:1-C,611-15-4,RU18.1_7788,245.0,2.389166
2,2,298.15,815.2,0.416,C-C-C(-C)-C-O,34713-94-5,RU18.1_7104,416.0,2.619093
3,3,293.15,2246.4,2.530,Br/C=C\Br,590-11-4,RU18.1_2945,2530.0,3.403121
4,4,293.15,833.5,1.480,C-C-C-C-C-C=O,66-25-1,RU18.1_5830,1480.0,3.170262
...,...,...,...,...,...,...,...,...,...
793,793,293.15,1384.2,9.870,O-C-C(-F)(-F)-F,75-89-8,RU18.1_10376,9870.0,3.994317
794,794,290.15,816.7,54.300,C#C-C#N,1070-71-9,RU18.1_2511,54300.0,4.734800
795,795,293.15,853.0,0.253,C-C(-C)-C-O-C(=O)-C-C(-C)-C,589-59-3,RU18.1_6401,253.0,2.403121
796,796,282.15,1405.0,187.000,F-C(-Cl)-Cl,75-43-4,RU18.1_3193,187000.0,5.271842


In [12]:
#make smiles canonical and standardized
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
df_crc['standardized_smiles'] = df_crc.apply(lambda row: Chem.MolToSmiles(rdMolStandardize.ChargeParent(Chem.MolFromSmiles(row['canonical_smiles']))), axis=1)
#save to csv
df_crc.to_csv('/localhome/cschiebroek/MDFPs/mdfptools/carl/data_curation/CRC_standardized_not_cleaned.csv')

[15:52:55] Initializing MetalDisconnector
[15:52:55] Running MetalDisconnector
[15:52:55] Initializing Normalizer
[15:52:55] Running Normalizer
[15:52:55] Running LargestFragmentChooser
[15:52:55] Fragment: CCCC(CC)CC
[15:52:55] New largest fragment: CCCC(CC)CC (26)
[15:52:55] Running Uncharger
[15:52:55] Initializing MetalDisconnector
[15:52:55] Running MetalDisconnector
[15:52:55] Initializing Normalizer
[15:52:55] Running Normalizer
[15:52:55] Initializing MetalDisconnector
[15:52:55] Running MetalDisconnector
[15:52:55] Initializing Normalizer
[15:52:55] Running Normalizer
[15:52:55] Running LargestFragmentChooser
[15:52:55] Fragment: C=Cc1ccccc1C
[15:52:55] New largest fragment: C=Cc1ccccc1C (19)
[15:52:55] Running Uncharger
[15:52:55] Initializing MetalDisconnector
[15:52:55] Running MetalDisconnector
[15:52:55] Initializing Normalizer
[15:52:55] Running Normalizer
[15:52:55] Initializing MetalDisconnector
[15:52:55] Running MetalDisconnector
[15:52:55] Initializing Normalizer
[1

In [77]:
df_fragrance_db = pd.read_csv('/localhome/cschiebroek/ShuZe/vp/data cleaning/fragrancedb.tsv', sep = "\t")
#drop where raw_vp_pubchem is nan
df_fragrance_db = df_fragrance_db.dropna(subset=['raw_vp_pubchem'])
#if there is a ";" in the raw_vp_pubchem, split over rows
df_fragrance_db = df_fragrance_db[df_fragrance_db['raw_vp_pubchem'].str.contains(';')]
df_fragrance_db = df_fragrance_db.reset_index(drop=True)
#split over rows
df_fragrance_db = df_fragrance_db.assign(raw_vp_pubchem=df_fragrance_db['raw_vp_pubchem'].str.split(';')).explode('raw_vp_pubchem')
#drop where raw_vp_pubchem is nan
df_fragrance_db
#drop if no '°' in raw_vp_pubchem
df_fragrance_db = df_fragrance_db[df_fragrance_db['raw_vp_pubchem'].str.contains('°')]
#make column with unit: kPa, Pa, mm Hg if in the string, else None. 
df_fragrance_db['unit'] = df_fragrance_db['raw_vp_pubchem'].apply(lambda x: 'kPa' if 'kPa' in x else ('Pa' if 'Pa' in x else ('mm Hg' if 'mm Hg' in x else ('mm Hg' if 'mmHg' in x else ('mm Hg' if 'MM HG' in x else None)))))
#count
df_fragrance_db['unit'].value_counts()
df_fragrance_db
#now extract the value: if there is a ":" in the string, take the value after the ":", else take the value before the unit
df_fragrance_db['raw_vp_value'] = df_fragrance_db.apply(lambda row: row['raw_vp_pubchem'].split(':')[1] if ':' in row['raw_vp_pubchem'] else row['raw_vp_pubchem'].split(row['unit'])[0], axis=1)
#if 'Vapor pressure = ' in raw_vp_value, remove it
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].str.replace('Vapor pressure = ', '')
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].str.replace('vapor pressure = ', '')

#if mm Hg in raw_vp_value, remove it
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].str.replace('mm Hg', '')
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].str.replace('mmHg', '')
#handle cases with 5.02X10-2, to 5.02e-2
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].str.replace('X10', 'e')
df_fragrance_db.head(50)
#check if there are some that cant be converted to float
#drop if 'more' or 'less' in raw_vp_value
df_fragrance_db = df_fragrance_db[~df_fragrance_db['raw_vp_value'].str.contains('more')]
df_fragrance_db = df_fragrance_db[~df_fragrance_db['raw_vp_value'].str.contains('less')]
df_fragrance_db = df_fragrance_db[~df_fragrance_db['raw_vp_value'].str.contains('negligible')]
df_fragrance_db = df_fragrance_db[~df_fragrance_db['raw_vp_value'].str.contains('est')]
df_fragrance_db = df_fragrance_db[~df_fragrance_db['raw_vp_value'].str.contains('solution')]


#if hPa in raw_vp_pubchem, and unit is Pa, change unit to hPa
df_fragrance_db['unit'] = df_fragrance_db.apply(lambda row: 'hPa' if 'hPa' in row['raw_vp_pubchem'] else row['unit'], axis=1)
#remove h from raw_vp_value for these
df_fragrance_db['raw_vp_value'] = df_fragrance_db.apply(lambda row: row['raw_vp_value'].replace('h', '') if 'hPa' in row['raw_vp_pubchem'] else row['raw_vp_value'], axis=1)
#show where hPA in raw_vp_pubchem
df_fragrance_db[df_fragrance_db['raw_vp_pubchem'].str.contains('hPa')]

import re
#if at * °C in raw_vp_value, remove it
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].apply(lambda x: re.sub(r'at (.*)°C', '', x))
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].apply(lambda x: re.sub(r'@ (.*)°C', '', x))
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].apply(lambda x: re.sub(r'(.*)°C at', '', x))
#drop if just '' in raw_vp_value
df_fragrance_db = df_fragrance_db[df_fragrance_db['raw_vp_value'] != '']


vals = df_fragrance_db['raw_vp_value'].values
to_inspect_vals = []
for v in vals:
    try:
        float(v)
    except:
        print(v)
        to_inspect_vals.append(v)
#show these
print(len(to_inspect_vals))
df_fragrance_db[df_fragrance_db['raw_vp_value'].isin(to_inspect_vals)]
#drop these
df_fragrance_db = df_fragrance_db[~df_fragrance_db['raw_vp_value'].isin(to_inspect_vals)]
#convert to float
df_fragrance_db['raw_vp_value'] = df_fragrance_db['raw_vp_value'].astype(float)
df_fragrance_db.head(50)
#drop where unit is none
df_fragrance_db = df_fragrance_db.dropna(subset=['unit'])
df_fragrance_db.head(50)

0.02  at 77°F
10  at 99.1°F
46 MM HG 
 0.0637   /l-Menthol/
 12.7 Pa 
40  at 50.9°F
40.6 MM HG 
1 MM HG 
1 MM HG AT 91 °C
2.8 MM HG 
0.21 MM HG 
 10 Pa 
0.6   (0.89 
 1 Pa
 
Vapor
10  at 245°F
0.0002  at 77°F
0.1 @68°F
0.6  at 77°F
1 MM HG 
 0.0025 MM HG AT 50 °C
0.003  at 122°F
23


,Unnamed: 0,smiles,name,cid,raw_vp_pubchem,unit,raw_vp_value
0,1,C1=CC2=C(C=C1)C=CC=C2,FrDB_2,931.0,0.05 mm Hg at 68 °F,mm Hg,0.0500
0,1,C1=CC2=C(C=C1)C=CC=C2,FrDB_2,931.0,"1 mm Hg at 126.7° F (NTP, 1992)",mm Hg,1.0000
0,1,C1=CC2=C(C=C1)C=CC=C2,FrDB_2,931.0,0.085 mm Hg at 25 °C,mm Hg,0.0850
0,1,C1=CC2=C(C=C1)C=CC=C2,FrDB_2,931.0,"Vapor pressure, Pa at 25 °C: 11",Pa,11.0000
1,4,C1=CC=CC=C1,FrDB_5,241.0,60 mm Hg at 59 °F,mm Hg,60.0000
1,4,C1=CC=CC=C1,FrDB_5,241.0,"76 mm Hg at 68° F (NTP, 1992)",mm Hg,76.0000
1,4,C1=CC=CC=C1,FrDB_5,241.0,94.8 mm Hg at 25 °C,mm Hg,94.8000
1,4,C1=CC=CC=C1,FrDB_5,241.0,"Vapor pressure, kPa at 20 °C: 10",kPa,10.0000
2,5,C1CCC=CC1,FrDB_6,8079.0,89.0 mm Hg at 25 °C (est),mm Hg,89.0000
2,5,C1CCC=CC1,FrDB_6,8079.0,"Vapor pressure, kPa at 20 °C: 8.9",kPa,8.9000


In [25]:
df_fragrance_db = pd.read_csv('/localhome/cschiebroek/ShuZe/vp/data cleaning/fragrancedb.tsv', sep = "\t")
raw_vp_pubchem = df_fragrance_db['raw_vp_pubchem'].tolist()
raw_vp_pubchem[-1]
for i in raw_vp_pubchem:
    #if there is no degrees sign, print
    if '°' not in str(i):
        print(i)

nan
nan
nan
23.52 mmHg
nan
nan
nan
nan
158.02 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.82 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.00 mmHg
nan
nan
nan
nan
nan
0.82 mmHg
0.06 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1 mm Hg (NIOSH, 2016);2.25 mmHg;1 mmHg;1 mmHg
nan
nan
nan
nan
nan
5.12X10-4 mm Hg, temp not specified
nan
nan
0.53 mmHg
nan
nan
nan
nan
0.36 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1.50 mmHg
nan
nan
nan
nan
24.89 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.01 mmHg;0.0055 mm Hg
nan
nan
nan
8.06 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
5.08 mmHg
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.61 mmHg
0.91 mmHg
3.00 mmHg
3.04 mmHg
0.94 mmHg
n